# Practice 03

Today, we will compute various properties of k-mers in a DNA sequence, notably using De Brujin graphs: https://en.wikipedia.org/wiki/De_Bruijn_graph

(additional information, not required for the exam: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5531759/)

# Q1

Using pandas, load the content of `../data/dna_adj.csv`, and output the following:
- number of elements equal to 0
- the 98th percentile of values in the matrix
- the sum of rows with a sequence starting with 'TT'

Example output:
`(1567,
 200.0,
 TTG    1.0
 TTC    7.0
 TTA    2.0
 TTT    3.0
 dtype: float64)`

In [1]:
import pandas as pd
dna_adj_file = "./data/dna_adj.csv"
df = pd.read_csv(dna_adj_file, index_col=0)

In [3]:
# number of elements equal to 0
df[df == 0.0].count().sum()

3861

In [4]:
# the 98th percentile of values in the matrix
df.quantile(0.98).sum()

490.38000000000034

In [5]:
# the sum of rows with a sequence starting with 'TT'
df.loc[[i for i in df.index if i.startswith("TT")]].sum(axis=1)

TTA     3.0
TTC    29.0
TTT     5.0
TTG    17.0
dtype: float64

## Q2a

You are asked to write a graph checker class. Write a python class that has the following characteristics:
- it has an `__init__()` method
- the aforementioned method takes a networkx graph as an argument, and keeps it as a data attribute when creating an instance
- it has a `check()` method
- this methods returns false if the passed graph has no node, and true if it has at least one node.



In [6]:
## Write your class below
class GraphChecker:
    def __init__(self, g):
        self.g = g
        
    def check(self):
        n = self.g.number_of_nodes()
        if n != 0:
            return True
        else:
            return False

## The code here should run, assuming you wrote the class above properly, and named it GraphChecker
import networkx as nx
g = nx.Graph()

gc = GraphChecker(g)
if gc.check():
    print("This graph has nodes!")
else:
    print("Warning! This graph has no nodes!")

Warning! This graph has no nodes!


## Q2b

Create a new `check_error()` method in your class, that raises an error if the graph is empty.

Write code below to create a new instance of your graph checker class, test the graph `g` with the new `check_error()` method, catches the error, print the number of nodes in the graph, then re-raises the error.

In [7]:
class GraphChecker:
    def __init__(self, g):
        self.g = g
        
    def check(self):
        n = self.g.number_of_nodes()
        if n != 0:
            return True
        else:
            return False
        
    def check_error(self):
        flag = self.check()
        if not flag:
            print(f"Number of nodes in graph: {self.g.number_of_nodes()}")
            print("Warning! This graph has no nodes!")
            
g = nx.Graph()
gc = GraphChecker(g)
gc.check_error()

Number of nodes in graph: 0
Warning! This graph has no nodes!


## Q3a

Create a De Brujin graph for DNA sequences of length 3. The graph should have the following properties:
- each node correspond to a 3-letter long sequence (e.g. ATT, TTC, CTG)
- there should be an edge from one node to another if the initial node's last two letters match the final node's first two letters. (e.g., there is an edge from TTA to TAC, but not from TTA to TTC)
- only the letters A, T, G, C need be considered

Output: an object that can be used as a De Brujin graph (e.g., a networkx graph), with the right network nodes and topology.

In [8]:
def deBrujinGraph(sequence, k=3):
    g = nx.MultiDiGraph()
    letters = {'A', 'C', 'G', 'T'}
    nodes = [sequence[i:i+k] for i in range(len(sequence)-k+1) if set(sequence[i:i+k]).issubset(letters)]
    for i in range(len(nodes)-1):
        g.add_edge(nodes[i], nodes[i+1])
    return g

## Q3b

Using the content from `dna.txt`, and the previously created De Brujin graph, count the number of 3-mers in each sequence, as well as the number of transitions between them.

What edge has the highest count? Give the starting and final node, as well as the count associated with that edge.

Example output: `count=5 nodes=('GCT', 'CTA')`

In [9]:
dna_file = "./data/dna.txt"
with open(dna_file, "r") as f:
    dna = f.read().replace("\n", "")
    
print(dna)

AATCACGTTATGTATACTTCTCGTCGGTCCCCCTCGTCCGTCGGTTTTCTCGTTGCCGCTGTTGCCGTCTCACGGGAGCCCGCTCGTTCGATTACTCCTCGTGGTGGCGCGTGCGAGCGATCCGCGGCACGCCTTCGACCGTCACGGTCTCGCCGAGGATGTACTTCGACGCGGGACTCGCGAGGAACTGCGCGACGGAGGCGATCTCGTCCGGAACGCCGATCTGTCGATCGACCTCCTCGAGGTCGATCTCGTCGGCGCTGATACCCATCTGACTCTCGAGGCCCTCGGTGACGATCAGTCCGGGCATGATCCCGTTGACACGGACCCCGTACTCCGCCCACTCGTAGGCGAGCGTCCGCGTCAGGTTGTTCATCCCGGCTTTCGACGCGGCGTAGTGGGACATCTGCGGCGCGCCGTCGCGGCCGGCCACGGAGGAGATGTTGATGACCGTCCCGCTGCCGGACTCGCGCATGTACTCGCCGACGACCTGTGTGCAGTTGAAGGTGCCGTTGAGGTTGATGTCGACGATCGTTCGCCAGGCGTTCTGGCTGAACTCCTCGAACGGAGCCTGGAAACTCGCGCCCGCGTTGTTGACCAGAATGTCGATTCCGCCGAACTCGTCGACGGTCGCCTCGGCGAGCGCTTCCACCGCGTCCCAGTCGGTGATATCGCACTCGACGGCGAGCGCGGTCCCCGGCCGGTCGCTGTCGTTGATCGCTCCCGCGACCTCCTCGAGGTCCTCGAGCGAGCGGGAGCAGACGACGACGTCGACACCGTCGGCCGCGAACCGTTCCGCGATCGCTCGACCGATGCCGGTCGACGCCCCGGTGACGATCGCCGTCTCTCCCGATACCTCGTAGTCAGTAGTCGTCATGGTGTGACAACACATACGAAGAAAACACTTGTAAATCCGATTGTTTCTCCACTGATAGGTCGGGCGCGGTGCTACGGTTCGGTGGACAAGATCGAGTATGGAGAATTCGGGTGGATATCCGCT

In [10]:
g = deBrujinGraph(dna)
notrans = sum([v[1] for v in g.degree])
print(f"Number of 3-mers in the sequence: {len(g)}")
print(f"Number of transitions: {notrans}")

highest_node = {}
for edge in g.edges:
    highest_node[edge[:-1]] = highest_node.get(edge[:-1], 0) + 1
    
highest_node = sorted(highest_node.items(), key=lambda x:x[1], reverse=True)[0]
print(f"count={highest_node[1]} nodes=({highest_node[0][0]}, {highest_node[0][1]})")

Number of 3-mers in the sequence: 64
Number of transitions: 4074
count=47 nodes=(GTC, TCG)


# Q4 (hardest question, try last!)

Find the largest value k of the De Brujin graph where at least one edge has a value greater than one (e.g., you observe the same transition more than once), using the penguinpox genome stored in `../data/penguinpox.fasta`

Print k, and as in Q2b, print the edge count, and connecting nodes

Example output: `k=13, count=2, nodes=('GCTG', 'GCTA')`

In [11]:
penguinpox_file = "./data/penguinpox.fasta"
with open(penguinpox_file, "r") as f:
    header = f.readline()
    pox = f.read().replace("\n", "")
    
print(pox)

...


In [12]:
g = deBrujinGraph(pox, k=4)
ne_dict = {}
for node, v in g.degree:
    temp = {}
    for edge in g.edges(node):
        temp[edge]= temp.get(edge, 0) + 1
    temp = {k : v for k, v in temp.items() if v>1}
    ne_dict.update({node: temp})
        
calc = {'k': set(), 'node': "", 'count': 0}
for node, pair in ne_dict.items():
    for n in pair.keys():
        calc['k'].add(n)
    p = sorted(pair.items(), key=lambda x: x[1], reverse=True)[0]
    if p[1] > calc['count']:
        calc['node'] = p[0]
        calc['count'] = p[1]
        
print(f"k={len(calc['k'])}, count={calc['count']}, nodes={calc['node']}")

k=1023, count=763, nodes=('TATA', 'ATAT')
